In [1]:
# Re-import required dependencies due to kernel reset
from typing import List, Tuple, Dict, Set, Union, Optional
from queue import PriorityQueue
import itertools

In [2]:
class Fluent(object):
    def __init__(self, name: str, *args: str, negated: bool = False):
        self.name = name
        self.args = args
        self.negated = negated

    def __str__(self) -> str:
        prefix = "not " if self.negated else ""
        return f"{prefix}{self.name} {' '.join(self.args)}"

    def __repr__(self) -> str:
        return f"Fluent<{self}>"

    def __hash__(self) -> int:
        return hash((self.name, self.args, self.negated))

    def __eq__(self, other: object) -> bool:
        return (
            isinstance(other, Fluent) and
            self.name == other.name and
            self.args == other.args and
            self.negated == other.negated
        )

    def __invert__(self) -> 'Fluent':
        return Fluent(self.name, *self.args, negated=not self.negated)

    def positive(self) -> 'Fluent':
        return Fluent(self.name, *self.args, negated=False)


class ActiveFluents(object):
    def __init__(self, fluents: Optional[Set[Fluent]] = None):
        if any(fluent.negated for fluent in fluents):
            raise ValueError("All fluents in active fluents must be positive.")
        self.fluents: Set[Fluent] = set(fluents) if fluents else set()

    def update(self, fluents: Set[Fluent]):
        """Apply fluents to the active set: add positives, remove targets of negations."""
        positives = {f for f in fluents if not f.negated}
        negatives = {f.positive() for f in fluents if f.negated}
        self.fluents = self.fluents - negatives | positives

    def copy(self) -> 'ActiveFluents':
        return ActiveFluents(set(self.fluents))

    def __contains__(self, f: Fluent) -> bool:
        return f in self.fluents

    def __iter__(self):
        return iter(self.fluents)

    def __str__(self):
        return f"{{{', '.join(str(f) for f in sorted(self.fluents, key=str))}}}"

    def __repr__(self):
        return str(self)

    def __eq__(self, other: object) -> bool:
        return isinstance(other, ActiveFluents) and self.fluents == other.fluents

    def __hash__(self):
        return hash(frozenset(self.fluents))


class Effect:
    def __init__(self, time: float, resulting_fluents: Set[Fluent]):
        self.time = time
        self.resulting_fluents = resulting_fluents

    def __str__(self):
        rfs = ", ".join(str(f) for f in self.resulting_fluents)
        return f"after {self.time}: {rfs}"

    def __repr__(self):
        return f"Effect({self})"

    def __lt__(self, other: 'Effect') -> bool:
        return self.time < other.time


class ProbEffects(Effect):
    def __init__(self, time: float, prob_effects: List[Tuple[float, List[Effect]]], resulting_fluents: Set[Fluent] = set()):
        self.time = time
        self.prob_effects = prob_effects
        self.resulting_fluents = resulting_fluents

    def __str__(self):
        prob_lines = []
        for p, elist in self.prob_effects:
            outcomes = "; ".join(str(e) for e in elist)
            prob_lines.append(f"{p}: [{outcomes}]")
        return f"probabilistic after {self.time}: {{ {', '.join(prob_lines)} }}"

    def __repr__(self):
        return f"ProbEffects({self})"


class Action:
    def __init__(self, preconditions: List[Fluent], effects: List[Union[Effect, ProbEffects]], name: Optional[str] = None):
        self.preconditions = preconditions
        self.effects = effects
        self.name = name or "anonymous"

    def __str__(self):
        pre_str = ", ".join(str(p) for p in self.preconditions)
        eff_strs = []
        for eff in self.effects:
            if isinstance(eff, Effect):
                rfs = ", ".join(str(f) for f in eff.resulting_fluents)
                eff_strs.append(f"after {eff.time}: {rfs}")
            elif isinstance(eff, ProbEffects):
                prob_lines = []
                for p, elist in eff.prob_effects:
                    outcomes = []
                    for e in elist:
                        rf = ", ".join(str(f) for f in e.resulting_fluents)
                        outcomes.append(f"after {e.time}: {rf}")
                    prob_lines.append(f"\n        {p}: [{'; '.join(outcomes)}]")
                eff_strs.append(f"probabilistic after {eff.time}: {{ {', '.join(prob_lines)} }}")
        return f"Action('{self.name}'\n  Preconditions: [{pre_str}]\n  Effects:\n    " + "\n    ".join(eff_strs) + "\n)"

    def __repr__(self):
        return self.__str__()


class State:
    def __init__(self, time: float = 0, active_fluents: Optional[Set[Fluent]] = None, upcoming_effects: Optional[PriorityQueue] = None):
        self.time = time
        self.active_fluents = ActiveFluents(active_fluents)
        self.upcoming_effects = upcoming_effects or PriorityQueue()

    def satisfies_precondition(self, action: Action) -> bool:
        for p in action.preconditions:
            if p.negated:
                if p.positive() in self.active_fluents:
                    return False
            else:
                if p not in self.active_fluents:
                    return False
        return True

    def copy(self) -> 'State':
        new_queue = PriorityQueue()
        new_queue.queue = [x for x in self.upcoming_effects.queue]
        return State(
            time=self.time,
            active_fluents=self.active_fluents.copy().fluents,
            upcoming_effects=new_queue
        )

    def __hash__(self) -> int:
        return hash(self.time) + hash(self.active_fluents)

    def __eq__(self, other: object) -> bool:
        return isinstance(other, State) and hash(self) == hash(other)

    def __str__(self):
        return f"State<time={self.time}, active_fluents={self.active_fluents}>"

    def __repr__(self):
        return self.__str__()

In [7]:
!uv pip install ipytest
import ipytest
ipytest.autoconfig()

def test_active_fluents_update_1():
    f1 = Fluent("at", "r1", "roomA")
    f2 = Fluent("free", "r1")
    f3 = Fluent("holding", "r1", "medkit")
    not_f2 = ~f2
    not_f3 = ~f3

    af = ActiveFluents({f1, f2})
    print("Initial:", af)

    # Apply update with positive fluent and no conflict
    af.update({f3})
    assert f3 in af
    print("After adding f3:", af)

    # Apply update with negation of f2 (should remove f2)
    af.update({not_f2})
    assert f2 not in af
    assert f3 in af
    print("After removing f2:", af)

    # Apply update with both positive and negated fluent
    af.update({~f3, f2})
    assert f3 not in af
    assert f2 in af
    print("After removing f3 and re-adding f2:", af)

def test_active_fluents_update_2():
    af = ActiveFluents({
        Fluent('at', 'robot1', 'bedroom'),
        Fluent('free', 'robot1'),
    })

    upcoming_fluents = {
        ~Fluent('free', 'robot1'),
        ~Fluent('at', 'robot1', 'bedroom'),
        Fluent('at', 'robot1', 'kitchen'),
        ~Fluent('found', 'fork'),
    }

    af.update(upcoming_fluents)

    expected = ActiveFluents({
        Fluent('at', 'robot1', 'kitchen'),
    })

    assert af == expected, f"Unexpected result: {af}"

    # Now re-add a positive fluent
    af.update({Fluent('free', 'robot1')})

    expected = ActiveFluents({
        Fluent('free', 'robot1'),
        Fluent('at', 'robot1', 'kitchen'),
    })

    assert af == expected, f"Unexpected result after re-adding: {af}"


ipytest.run('-vv')

Using Python 3.10.12 environment at: /opt/.venv
Resolved 23 packages in 52ms                                         
Prepared 1 package in 5ms                                                
Installed 1 package in 1ms                                  
 + ipytest==0.14.2
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.5, pluggy-1.5.0 -- /opt/.venv/bin/python3
cachedir: .pytest_cache
metadata: {'Python': '3.10.12', 'Platform': 'Linux-6.8.0-52-generic-x86_64-with-glibc2.35', 'Packages': {'pytest': '8.3.5', 'pluggy': '1.5.0'}, 'Plugins': {'html': '4.1.1', 'timeout': '2.3.1', 'metadata': '3.1.1', 'anyio': '4.9.0'}}
rootdir: /notebooks
plugins: html-4.1.1, timeout-2.3.1, metadata-3.1.1, anyio-4.9.0
collecting ... collected 2 items

t_b5bb9cb3173f4f3ba72deab7a75c9d48.py::test_active_fluents_update_1 PASSED                   [ 50%]
t_b5bb9cb3173f4f3ba72deab7a75c9d48.py::test_active_fluents_update

<ExitCode.OK: 0>

In [8]:
class Operator:
    def __init__(self, name: str, parameters: List[Tuple[str, str]], preconditions: List[Fluent], effects: List[Union[Effect, ProbEffects]]):
        self.name = name
        self.parameters = parameters
        self.preconditions = preconditions
        self.effects = effects

    def instantiate(self, objects_by_type: Dict[str, List[str]]) -> List[Action]:
        grounded_actions = []
        domains = [objects_by_type[typ] for _, typ in self.parameters]
        for assignment in itertools.product(*domains):
            binding = {var: obj for (var, _), obj in zip(self.parameters, assignment)}
            if len(set(binding.values())) != len(binding):
                continue
            grounded_actions.append(self._ground(binding))
        return grounded_actions

    def _ground(self, binding: Dict[str, str]) -> Action:
        grounded_preconditions = [self._substitute_fluent(f, binding) for f in self.preconditions]
        grounded_effects = []
        for eff in self.effects:
            if isinstance(eff, ProbEffects):
                grounded_prob_effects = []
                for prob, effect_list in eff.prob_effects:
                    grounded_list = [
                        Effect(e.time, {self._substitute_fluent(f, binding) for f in e.resulting_fluents})
                        for e in effect_list
                    ]
                    grounded_prob_effects.append((prob, grounded_list))
                grounded_resulting_fluents = {self._substitute_fluent(f, binding) for f in eff.resulting_fluents}
                grounded_effects.append(ProbEffects(eff.time, grounded_prob_effects, grounded_resulting_fluents))
            elif isinstance(eff, Effect):
                grounded_fluents = {self._substitute_fluent(f, binding) for f in eff.resulting_fluents}
                grounded_effects.append(Effect(eff.time, grounded_fluents))
            
        name_str = f"{self.name} " + " ".join(binding[var] for var, _ in self.parameters)
        return Action(grounded_preconditions, grounded_effects, name=name_str)

    def _substitute_fluent(self, fluent: Fluent, binding: Dict[str, str]) -> Fluent:
        grounded_args = tuple(binding.get(arg, arg) for arg in fluent.args)
        return Fluent(fluent.name, *grounded_args, negated=fluent.negated)

def transition(state: State, action: Action) -> List[Tuple[State, float]]:
    if not state.satisfies_precondition(action):
        raise ValueError("Precondition not satisfied for applying action")

    new_state = state.copy()
    for effect in action.effects:
        new_state.upcoming_effects.put((new_state.time + effect.time, effect))

    # Fixme: is this necessary or can I just pass it to outcomes?
    outcomes: Dict[State, float] = {}
    _advance_to_terminal(new_state, prob=1.0, outcomes=outcomes)
    return list(outcomes.items())

def _advance_to_terminal(state: State, prob: float, outcomes: Dict[State, float]) -> None:
    while not state.upcoming_effects.empty():
        scheduled_time, effect = state.upcoming_effects.queue[0]

        # Check if we're ready to yield this state
        if scheduled_time > state.time and any(f.name == "free" for f in state.active_fluents):
            outcomes[state] = prob
            return

        # Advance time if necessary
        if scheduled_time > state.time:
            state.time = scheduled_time

        # Apply effect
        state.upcoming_effects.get()
        state.active_fluents.update(effect.resulting_fluents)

        if isinstance(effect, ProbEffects):
            for branch_prob, effects in effect.prob_effects:
                branched = state.copy()
                for e in effects:
                    branched.upcoming_effects.put((branched.time + e.time, e))
                _advance_to_terminal(branched, prob * branch_prob, outcomes)
            return  # stop after branching

    # No more effects; yield terminal state
    outcomes[state] = prob

def get_action_by_name(actions: List[Action], name: str) -> Action:
    for action in actions:
        if action.name == name:
            return action
    raise ValueError(f"No action found with name: {name}")

def get_next_actions(state: State, all_actions: List[Action]) -> List[Action]:
    # Step 1: Extract all `free(...)` fluents
    free_predicates = sorted(
        [f for f in state.active_fluents if f.name == "free" and not f.negated],
        key=str
    )
    neg_active_fluents = state.active_fluents.copy()
    neg_active_fluents.update({~p for p in free_predicates})

    # Step 2: Check each robot individually
    for free_pred in free_predicates:
        # Create a restricted version of the state
        temp_fluents = neg_active_fluents.copy()
        temp_fluents.update({free_pred})
        temp_state = State(time=state.time, active_fluents=temp_fluents)

        # Step 3: Check for applicable actions
        applicable = [a for a in all_actions if temp_state.satisfies_precondition(a)]
        if applicable:
            return applicable

    # Step 4: No applicable actions found
    return []

In [9]:
## Search


# Define the search operator again
search_op = Operator(
    name="search",
    parameters=[
        ("?robot", "robot"),
        ("?loc_from", "location"),
        ("?loc_to", "location"),
        ("?object", "object")
    ],
    preconditions=[
        Fluent("at", "?robot", "?loc_from"),
        ~Fluent("searched", "?loc_to", "?object"),
        Fluent("free", "?robot"),
        ~Fluent("found", "?object")
    ],
    effects=[
        Effect(
            time=0,
            resulting_fluents={
                ~Fluent("free", "?robot"),
                ~Fluent("at", "?robot", "?loc_from"),
                Fluent("searched", "?loc_to", "?object")
            }
        ),
        ProbEffects(
            time=5,
            resulting_fluents={
                Fluent("at", "?robot", "?loc_to"),
            },
            prob_effects=[
                (
                    0.8,
                    [
                        Effect(
                            time=0,
                            resulting_fluents={
                                Fluent("found", "?object")
                            }
                        ),
                        Effect(
                            time=3,
                            resulting_fluents={
                                Fluent("holding", "?robot", "?object"),
                                Fluent("free", "?robot")
                            }
                        )
                    ]
                ),
                (
                    0.2,
                    [
                        Effect(
                            time=0,
                            resulting_fluents={
                                Fluent("free", "?robot"),
                                ~Fluent("at", "?loc_to", "?object")
                            }
                        )
                    ]
                )
            ],
        )
    ],
)


# Ground a specific instance
objects_by_type = {
    "robot": ["r1", "r2"],
    "location": ["roomA", "roomB"],
    "object": ["cup", "bowl"]
}

search_actions = search_op.instantiate(objects_by_type)
print(search_actions)

# Define the initial state
search_action = search_actions[0]
initial_state = State(
    time=0,
    active_fluents={
        Fluent("at", "r1", "roomA"),
        Fluent("at", "r2", "roomB"),
        Fluent("free", "r1"),
        Fluent("free", "r2")
    }
)

print("== Initial State")
print(initial_state)

# Execute transition
print("== All available next actions")
action_1 = get_action_by_name(search_actions, 'search r1 roomA roomB cup')
action_2 = get_action_by_name(search_actions, 'search r2 roomB roomA bowl')
available_actions = get_next_actions(initial_state, search_actions)
for action in available_actions:
    print(action)

print("== State after one action")
outcomes = transition(initial_state, action_1)
assert len(outcomes) == 1
updated_state, _ = outcomes[0]

print("== All available next actions")
available_actions = get_next_actions(updated_state, search_actions)
for action in available_actions:
    print(action)

print("== State after another action")
outcomes = transition(updated_state, action_2)
for state, prob in outcomes:
    print(state, prob)
    print(state.upcoming_effects.queue)
    updated_state = state

[Action('search r1 roomA roomB cup'
  Preconditions: [at r1 roomA, not searched roomB cup, free r1, not found cup]
  Effects:
    after 0: searched roomB cup, not free r1, not at r1 roomA
    after 5: at r1 roomB
), Action('search r1 roomA roomB bowl'
  Preconditions: [at r1 roomA, not searched roomB bowl, free r1, not found bowl]
  Effects:
    after 0: searched roomB bowl, not free r1, not at r1 roomA
    after 5: at r1 roomB
), Action('search r1 roomB roomA cup'
  Preconditions: [at r1 roomB, not searched roomA cup, free r1, not found cup]
  Effects:
    after 0: not at r1 roomB, searched roomA cup, not free r1
    after 5: at r1 roomA
), Action('search r1 roomB roomA bowl'
  Preconditions: [at r1 roomB, not searched roomA bowl, free r1, not found bowl]
  Effects:
    after 0: not at r1 roomB, searched roomA bowl, not free r1
    after 5: at r1 roomA
), Action('search r2 roomA roomB cup'
  Preconditions: [at r2 roomA, not searched roomB cup, free r2, not found cup]
  Effects:
    af

In [10]:
# Ground a specific instance
objects_by_type = {
    "robot": ["r1"],
    "location": ["roomA", "roomB"],
    "object": ["cup", "bowl"]
}

search_actions = search_op.instantiate(objects_by_type)
print(search_actions)

# Define the initial state
search_action = search_actions[0]
initial_state = State(
    time=0,
    active_fluents={
        Fluent("at", "r1", "roomA"),
        Fluent("free", "r1"),
    }
)

print("== Initial State")
print(initial_state)

# Execute transition
print("== All available next actions")
action_1 = get_action_by_name(search_actions, 'search r1 roomA roomB cup')
action_2 = get_action_by_name(search_actions, 'search r1 roomB roomA bowl')
available_actions = get_next_actions(initial_state, search_actions)
for action in available_actions:
    print(action)

print("== State after one action")
outcomes = transition(initial_state, action_1)
for state, prob in outcomes:
    print(state)
updated_state, _ = outcomes[0]

print("== All available next actions")
available_actions = get_next_actions(updated_state, search_actions)
for action in available_actions:
    print(action)

print("== State after another action")
outcomes = transition(updated_state, action_2)
for state, prob in outcomes:
    print(state, prob)
    print(state.upcoming_effects.queue)
    updated_state = state

[Action('search r1 roomA roomB cup'
  Preconditions: [at r1 roomA, not searched roomB cup, free r1, not found cup]
  Effects:
    after 0: searched roomB cup, not free r1, not at r1 roomA
    after 5: at r1 roomB
), Action('search r1 roomA roomB bowl'
  Preconditions: [at r1 roomA, not searched roomB bowl, free r1, not found bowl]
  Effects:
    after 0: searched roomB bowl, not free r1, not at r1 roomA
    after 5: at r1 roomB
), Action('search r1 roomB roomA cup'
  Preconditions: [at r1 roomB, not searched roomA cup, free r1, not found cup]
  Effects:
    after 0: not at r1 roomB, searched roomA cup, not free r1
    after 5: at r1 roomA
), Action('search r1 roomB roomA bowl'
  Preconditions: [at r1 roomB, not searched roomA bowl, free r1, not found bowl]
  Effects:
    after 0: not at r1 roomB, searched roomA bowl, not free r1
    after 5: at r1 roomA
)]
== Initial State
State<time=0, active_fluents={at r1 roomA, free r1}>
== All available next actions
Action('search r1 roomA roomB c

In [11]:
## Move (seems to work)

# Define the search operator again
move_op = Operator(
    name="move",
    parameters=[
        ("?robot", "robot"),
        ("?loc_from", "location"),
        ("?loc_to", "location"),
    ],
    preconditions=[
        Fluent("at", "?robot", "?loc_from"),
        Fluent("free", "?robot"),
    ],
    effects=[
        Effect(
            time=0,
            resulting_fluents={
                ~Fluent("free", "?robot"),
            }
        ),
        Effect(
            time=5,
            resulting_fluents={
                Fluent("free", "?robot"),
                ~Fluent("at", "?robot", "?loc_from"),
                Fluent("at", "?robot", "?loc_to"),
            }
        )]
)

# Ground a specific instance
objects_by_type = {
    "robot": ["r1", "r2"],
    "location": ["roomA", "roomB"],
    "object": ["cup", "bowl"]
}
move_actions = move_op.instantiate(objects_by_type)
for action in move_actions:
    print(action)

# Define the initial state
action = move_actions[0]
initial_state = State(
    time=0,
    active_fluents={
        Fluent("at", "r1", "roomA"),
        Fluent("at", "r2", "roomA"),
        Fluent("free", "r1"),
        Fluent("free", "r2")
    }
)


print("== Initial State")
print(initial_state)

# Execute transition
print("== All available next actions")
available_actions = get_next_actions(initial_state, move_actions)
for action in available_actions:
    print(action)

print("== State after one action")
outcomes = transition(initial_state, available_actions[0])
for state, prob in outcomes:
    print(state, prob)
    print(state.upcoming_effects.queue)
    updated_state = state

print("== All available next actions")
available_actions = get_next_actions(updated_state, move_actions)
for action in available_actions:
    print(action)

print("== State after another action")
outcomes = transition(updated_state, available_actions[0])
for state, prob in outcomes:
    print(state, prob)
    print(state.upcoming_effects.queue)
    updated_state = state

Action('move r1 roomA roomB'
  Preconditions: [at r1 roomA, free r1]
  Effects:
    after 0: not free r1
    after 5: at r1 roomB, free r1, not at r1 roomA
)
Action('move r1 roomB roomA'
  Preconditions: [at r1 roomB, free r1]
  Effects:
    after 0: not free r1
    after 5: not at r1 roomB, free r1, at r1 roomA
)
Action('move r2 roomA roomB'
  Preconditions: [at r2 roomA, free r2]
  Effects:
    after 0: not free r2
    after 5: not at r2 roomA, free r2, at r2 roomB
)
Action('move r2 roomB roomA'
  Preconditions: [at r2 roomB, free r2]
  Effects:
    after 0: not free r2
    after 5: at r2 roomA, free r2, not at r2 roomB
)
== Initial State
State<time=0, active_fluents={at r1 roomA, at r2 roomA, free r1, free r2}>
== All available next actions
Action('move r1 roomA roomB'
  Preconditions: [at r1 roomA, free r1]
  Effects:
    after 0: not free r1
    after 5: at r1 roomB, free r1, not at r1 roomA
)
== State after one action
State<time=0, active_fluents={at r1 roomA, at r2 roomA, free r

In [12]:
search_op = Operator(
    name="search",
    parameters=[
        ("?robot", "robot"),
        ("?loc_from", "location"),
        ("?loc_to", "location"),
        ("?object", "object")
    ],
    preconditions=[
        Fluent("at", "?robot", "?loc_from"),
        ~Fluent("searched", "?loc_to", "?object"),
        Fluent("free", "?robot"),
        ~Fluent("found", "?object")
    ],
    effects=[
        Effect(
            time=0,
            resulting_fluents={
                ~Fluent("free", "?robot"),
                ~Fluent("at", "?robot", "?loc_from"),
                Fluent("searched", "?loc_to", "?object")
            }
        ),
        ProbEffects(
            time=5,
            resulting_fluents={
                Fluent("at", "?robot", "?loc_to"),
            },
            prob_effects=[
                (0.8, [Effect(time=0, resulting_fluents={Fluent("found", "?object")}),
                       Effect(time=3, resulting_fluents={
                                Fluent("holding", "?robot", "?object"),
                                Fluent("free", "?robot")})]),
                (
                    0.2,
                    [
                        Effect(
                            time=0,
                            resulting_fluents={
                                Fluent("free", "?robot"),
                                ~Fluent("at", "?loc_to", "?object")
                            }
                        )
                    ]
                )
            ],
        )
    ],
)

def test_move_sequence():
    # Define move operator
    move_op = Operator(
        name="move",
        parameters=[
            ("?robot", "robot"),
            ("?loc_from", "location"),
            ("?loc_to", "location"),
        ],
        preconditions=[
            Fluent("at", "?robot", "?loc_from"),
            Fluent("free", "?robot"),
        ],
        effects=[
            Effect(
                time=0,
                resulting_fluents={
                    ~Fluent("free", "?robot"),
                }
            ),
            Effect(
                time=5,
                resulting_fluents={
                    Fluent("free", "?robot"),
                    ~Fluent("at", "?robot", "?loc_from"),
                    Fluent("at", "?robot", "?loc_to"),
                }
            )
        ]
    )

    # Ground actions
    objects_by_type = {
        "robot": ["r1", "r2"],
        "location": ["roomA", "roomB"],
    }
    move_actions = move_op.instantiate(objects_by_type)

    # Initial state
    initial_state = State(
        time=0,
        active_fluents={
            Fluent("at", "r1", "roomA"),
            Fluent("at", "r2", "roomA"),
            Fluent("free", "r1"),
            Fluent("free", "r2")
        }
    )

    # First transition: move r1 from roomA to roomB
    available = get_next_actions(initial_state, move_actions)
    assert any(a.name == "move r1 roomA roomB" for a in available)
    a1 = get_action_by_name(available, "move r1 roomA roomB")
    outcomes = transition(initial_state, a1)
    assert len(outcomes) == 1
    state1, prob1 = outcomes[0]
    assert prob1 == 1.0
    assert Fluent("free", "r1") not in state1.active_fluents
    assert Fluent("free", "r2") in state1.active_fluents
    assert len(state1.upcoming_effects.queue) == 1

    # Second transition: move r2 from roomA to roomB
    available = get_next_actions(state1, move_actions)
    assert any(a.name == "move r2 roomA roomB" for a in available)
    a2 = get_action_by_name(available, "move r2 roomA roomB")
    outcomes = transition(state1, a2)
    assert len(outcomes) == 1
    state2, prob2 = outcomes[0]
    assert prob2 == 1.0
    assert Fluent("at", "r1", "roomB") in state2.active_fluents
    assert Fluent("at", "r2", "roomB") in state2.active_fluents
    assert Fluent("free", "r1") in state2.active_fluents
    assert Fluent("free", "r2") in state2.active_fluents
    assert state2.time == 5
    assert len(state2.upcoming_effects.queue) == 0


def test_search_sequence():
    # Define objects
    objects_by_type = {
        "robot": ["r1"],
        "location": ["roomA", "roomB"],
        "object": ["cup", "bowl"]
    }

    # Ground actions
    search_actions = search_op.instantiate(objects_by_type)

    # Initial state
    initial_state = State(
        time=0,
        active_fluents={
            Fluent("at", "r1", "roomA"),
            Fluent("free", "r1"),
        }
    )

    # Select action: search r1 roomA roomB cup
    action_1 = get_action_by_name(search_actions, 'search r1 roomA roomB cup')
    outcomes = transition(initial_state, action_1)

    # Assert both probabilistic outcomes exist
    assert len(outcomes) == 2
    probs = {round(p, 2) for _, p in outcomes}
    assert probs == {0.8, 0.2}

    # Verify high-probability (success) branch
    high_prob_state = next(s for s, p in outcomes if round(p, 2) == 0.8)
    assert Fluent("at", "r1", "roomB") in high_prob_state.active_fluents
    assert Fluent("holding", "r1", "cup") in high_prob_state.active_fluents
    assert Fluent("free", "r1") in high_prob_state.active_fluents
    assert Fluent("found", "cup") in high_prob_state.active_fluents
    assert Fluent("searched", "roomB", "cup") in high_prob_state.active_fluents
    assert high_prob_state.time == 8

    # Verify low-probability (failure to find object) branch
    low_prob_state = next(s for s, p in outcomes if round(p, 2) == 0.2)
    assert Fluent("at", "r1", "roomB") in low_prob_state.active_fluents
    assert Fluent("free", "r1") in low_prob_state.active_fluents
    assert Fluent("found", "cup") not in low_prob_state.active_fluents
    assert Fluent("holding", "r1", "cup") not in low_prob_state.active_fluents
    assert Fluent("searched", "roomB", "cup") in low_prob_state.active_fluents
    assert low_prob_state.time == 5

    # Continue from high-probability outcome
    action_2 = get_action_by_name(search_actions, 'search r1 roomB roomA bowl')
    next_outcomes = transition(high_prob_state, action_2)

    assert len(next_outcomes) == 2
    for state, prob in next_outcomes:
        assert Fluent("at", "r1", "roomA") in state.active_fluents
        assert Fluent("searched", "roomA", "bowl") in state.active_fluents
        assert Fluent("found", "cup") in state.active_fluents
        assert Fluent("holding", "r1", "cup") in state.active_fluents
        if round(prob, 2) == 0.8:
            assert Fluent("found", "bowl") in state.active_fluents
            assert Fluent("holding", "r1", "bowl") in state.active_fluents
            assert Fluent("free", "r1") in state.active_fluents
            assert state.time == 16
        elif round(prob, 2) == 0.2:
            assert Fluent("found", "bowl") not in state.active_fluents
            assert Fluent("holding", "r1", "bowl") not in state.active_fluents
            assert Fluent("free", "r1") in state.active_fluents
            assert state.time == 13


test_move_sequence()
test_search_sequence()